#### String Output Parser

In [4]:
import openai
from openai import OpenAI
import langchain
import os
# Step 1: Load environment variables
%reload_ext dotenv
%dotenv
# Step 3: Access your OpenAI API key from environment variables
for key, value in os.environ.items():
    if key == "OPENAI_API_KEY":
        print(f"{key}: value")

OPENAI_API_KEY: value


In [5]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [194]:
from langchain_openai.chat_models import ChatOpenAI
chat = ChatOpenAI(model='gpt-4o-mini', temperature=0,max_completion_tokens=50)
# response = chat.invoke("I have recently adopted a dog. Could you suggest some dog names?")
# print(response.content)

In [7]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser

In [8]:
hum_message = HumanMessage(content = "Can you give me an interesting fact I probably didn't know about?")

In [14]:
response = chat.invoke([hum_message])

In [15]:
response.content

'Sure! Did you know that honey never spoils? Archaeologists have found pots of honey in ancient'

In [16]:
type(response)

langchain_core.messages.ai.AIMessage

In [17]:
str_output_parser = StrOutputParser()

In [18]:
response_parsed = str_output_parser.invoke(response)
response_parsed

'Sure! Did you know that honey never spoils? Archaeologists have found pots of honey in ancient'

In [19]:
type(str_output_parser)

langchain_core.output_parsers.string.StrOutputParser

#### CommaSeparatedListOutputParser
Parses a language model's response into a list of strings, assuming the items are separated by commas.

In [42]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import ChatPromptTemplate,HumanMessagePromptTemplate

In [41]:
list_instructions = CommaSeparatedListOutputParser().get_format_instructions()
list_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [110]:
list_output_parser = CommaSeparatedListOutputParser()

In [111]:
hum_messag_template = HumanMessagePromptTemplate.from_template('''I adopted a {pet}, could you suggest three {pet} names?\n'''
                                                               +list_instructions)

In [112]:
chat_template = ChatPromptTemplate.from_messages([hum_messag_template])
chat_template

ChatPromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, template='I adopted a {pet}, could you suggest three {pet} names?\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'), additional_kwargs={})])

In [113]:
print(chat_template.messages[0])

prompt=PromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, template='I adopted a {pet}, could you suggest three {pet} names?\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`') additional_kwargs={}


In [114]:
chat_template_result = chat_template.invoke({'pet':'dog'})
chat_template_result

ChatPromptValue(messages=[HumanMessage(content='I adopted a dog, could you suggest three dog names?\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`', additional_kwargs={}, response_metadata={})])

In [115]:
chat_result = chat.invoke(chat_template_result)
chat_result

AIMessage(content='Buddy, Luna, Max', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 47, 'total_tokens': 53, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BS1RXJXchuHUWvDScveBh8xFO7Ypa', 'finish_reason': 'stop', 'logprobs': None}, id='run-fafef27c-3369-409e-be54-62ea9c0f1b08-0', usage_metadata={'input_tokens': 47, 'output_tokens': 6, 'total_tokens': 53, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [116]:
list_output_parser.invoke(chat_result)

['Buddy', 'Luna', 'Max']

### Using Chain

In [117]:
chain = chat_template | chat | list_output_parser
chain.invoke({'pet':'dog'})

['Buddy', 'Bella', 'Max']

In [51]:
inputs = [{"pet": "dog"}, {"pet": "cat"}, {"pet": "rabbit"}]
results = []
for input_data in inputs:
    results.append(chain.invoke(input_data))
print(results)

[['Buddy', 'Bella', 'Max'], ['Whiskers', 'Luna', 'Oliver'], ['Thumper', 'Nibbles', 'Flopsy']]


# Batching
In LangChain, batching refers to processing multiple inputs at once instead of one at a time

In [170]:
hum_messag_template = HumanMessagePromptTemplate.from_template('''I've recently adopted a {pet} which is a {breed}. Could you suggest several training tips?'''
                                                               +list_instructions)

In [171]:
chat_template = ChatPromptTemplate.from_messages([hum_messag_template])
chat_template

ChatPromptTemplate(input_variables=['breed', 'pet'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['breed', 'pet'], input_types={}, partial_variables={}, template="I've recently adopted a {pet} which is a {breed}. Could you suggest several training tips?Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`"), additional_kwargs={})])

In [172]:
chain = chat_template | chat | list_output_parser
response = chain.invoke({'pet':'dog','breed':'husky'})
response


['positive reinforcement',
 'consistency in commands',
 'socialization with other dogs and people',
 'regular exercise and mental stimulation',
 'patience and persistence',
 'using a firm but gentle tone',
 'establishing a routine',
 'teaching basic commands first',
 'avoiding harsh corrections',
 'providing plenty of toys and activities']

In [173]:
inputs = [{'pet':'dog','breed':'husky'},
           {'pet':'cat','breed':'siamy'}]

In [174]:
chain = chat_template | chat | list_output_parser
response = chain.batch(inputs=inputs)
response

[['positive reinforcement',
  'consistency in commands',
  'socialization with other dogs',
  'regular exercise',
  'mental stimulation activities',
  'patience during training',
  'establishing a routine',
  'using high-value treats',
  'teaching basic commands first',
  'avoiding harsh corrections'],
 ['positive reinforcement',
  'consistency',
  'clicker training',
  'short training sessions',
  'use treats',
  'establish a routine',
  'socialization',
  'patience',
  'avoid punishment',
  'provide mental stimulation']]

In [179]:
chain = chat_template | chat | list_output_parser
chain.batch([{'pet':'dog','breed':'betbol'},
           {'pet':'bird','breed':'macaw'}])

[['positive reinforcement',
  'consistency in commands',
  'short training sessions',
  'socialization with other dogs',
  'patience and understanding',
  'using high-value treats',
  'establishing a routine',
  'practicing basic commands',
  'gradually increasing distractions',
  'rewarding good behavior'],
 ['positive reinforcement',
  'consistency',
  'short training sessions',
  'use of treats',
  'patience',
  'socialization',
  'target training',
  'clicker training',
  'establish a routine',
  'avoid negative reinforcement']]

#### Streaming:
means the LLM's response is delivered token by token, instead of waiting for the full response
Think of ChatGPT typing in real time — that’s streaming.

In [1]:
chain = chat_template | chat
response = chain.stream({'pet':'dragon',
                         'breed':'night fury'})

NameError: name 'chat_template' is not defined

In [188]:
next(response)

AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-4492f231-906c-495a-9ccb-b61872678c25')

In [192]:
for i in response:
    print(i.content)

In [196]:
for i in response:
    print(i.content,end='')

In [197]:
type(chat_template)

langchain_core.prompts.chat.ChatPromptTemplate